# Hide API Keys before pushing to GitHub

In [1]:
import os
import json
import re

def replace_api_keys_in_notebooks(directory, keys):
    # Recursively find all .ipynb files in the directory
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.ipynb'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    notebook_content = json.load(f)

                # Replace API key values
                for cell in notebook_content['cells']:
                    if cell['cell_type'] == 'code':
                        new_source = []
                        for line in cell['source']:
                            for key in keys:
                                if key in line:
                                    line = replace_key_value(line, key)
                            new_source.append(line)
                        cell['source'] = new_source

                # Save the updated notebook
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(notebook_content, f, indent=2)

def replace_key_value(line, key):
    # Define patterns to find and replace the API keys
    patterns = [
        re.compile(rf'({key}\s*=\s*[\'"])(.*?)([\'"])'),  # Matches key="value" or key='value'
        re.compile(rf'([\'"]{key}[\'"]\s*:\s*[\'"])(.*?)([\'"])'),  # Matches 'key': 'value' or "key": "value"
        re.compile(rf'(\b{key}\b\s*=\s*)(.*?)(\s|$|#)'),  # Matches key=value
        re.compile(rf'(os\.environ\["{key}"\]\s*=\s*[\'"])(.*?)([\'"])'),  # Matches os.environ["key"] = "value"
        re.compile(rf'(os\.environ\["{key}"\s*=\s*[\'"])(.*?)([\'"])'),  # Matches os.environ["key="value"
    ]
    
    def replacer(match):
        full_match = match.group(0)
        value = match.group(2)
        return full_match.replace(value, '*' * len(value))
    
    for pattern in patterns:
        line = pattern.sub(replacer, line)
    
    return line

# Directory containing the notebooks
directory = '/home/ubuntu/efs-w210-capstone-ebs/05.GitHub_Project_Repo_GabbleGrid/06.Inference_and_Deployment'

# List of API keys to replace
keys = [
    'OPENAI_API_KEY',
    'SERPAPI_API_KEY',
    'api_key',
    'BING_API_KEY',
    'bing_api_key'
]

In [2]:
replace_api_keys_in_notebooks(directory, keys)